# After refactoring...

In [1]:
from pixasonics.core import App, Mapper
from pixasonics.features import MeanPixelValue
from pixasonics.synths import Theremin

# img_path = "/Users/balintl/Documents/GitHub/sonification/experiments/lsm_paper/figures/cellular_seq/Timepoint_001_220518-ST_C03_s1.jpg"
img_path = "test.jpg"

# Create app
app = App()
app.load_image(img_path) # Load image

# Create objects
mean_pix = MeanPixelValue()
theremin = Theremin()
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_high=1000)

# Add objects to app
app.add_feature(mean_pix)
app.add_synth(theremin)
app.add_mapper(pix2freq)


[Taichi] version 1.7.3, llvm 15.0.7, commit 5ec301be, osx, python 3.10.16


[I 01/30/25 11:05:26.942 28394824] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


MultiCanvas(height=580, width=1080)

IntSlider(value=50, description='Probe Width', layout=Layout(width='400px'), max=500, min=1, style=SliderStyle…

IntSlider(value=50, description='Probe Height', layout=Layout(width='400px'), max=500, min=1, style=SliderStyl…

...

Text(value='', description='MeanPixelValue_0_Min:', disabled=True, style=TextStyle(description_width='auto'))

Text(value='', description='MeanPixelValue_0_Max:', disabled=True, style=TextStyle(description_width='auto'))

Text(value='', description='MeanPixelValue_0_Last:', disabled=True, style=TextStyle(description_width='auto'))

Button(description='Reset MeanPixelValue MinMax', layout=Layout(height='40px', width='auto'), style=ButtonStyl…

Label(value='Mapper: MeanPixelValue ==> Theremin Frequency')

In [2]:
# test adding another mapping: mean pixel value to amplitude with an exponential curve
pix2amp = Mapper(mean_pix, theremin["amplitude"], exponent=2)
app.add_mapper(pix2amp)

Label(value='Mapper: MeanPixelValue ==> Theremin Amplitude')

In [3]:
pix2pan = Mapper(mean_pix, theremin["panning"])
app.add_mapper(pix2pan)

Label(value='Mapper: MeanPixelValue ==> Theremin Panning')

In [4]:
app.mappers

[Mapper: MeanPixelValue ==> Theremin Frequency,
 Mapper: MeanPixelValue ==> Theremin Amplitude,
 Mapper: MeanPixelValue ==> Theremin Panning]

In [5]:
import signalflow as sf
graph = sf.AudioGraph.get_shared_graph()
print(graph.structure)

 * audioout
   input0:
    * multiply
      input0:
       * bus
         input0:
          * stereo-panner
            input:
             * multiply
               input0:
                * sine
                  frequency:
                   * smooth
                     input:
                      * buffer-player
                        loop: 1.000000
                        rate: 1.000000
                     smooth: 0.999425
               input1:
                * smooth
                  input:
                   * buffer-player
                     loop: 1.000000
                     rate: 1.000000
                  smooth: 0.999425
            pan:
             * smooth
               input:
                * buffer-player
                  loop: 1.000000
                  rate: 1.000000
               smooth: 0.999425
      input1:
       * smooth
         input:
          * multiply
            input0:
             * decibels-to-amplitude
               input: 0.000000
   